## Example

Run the following kernel to import all necessary modules:

In [ ]:
include("../nonsmooth-forward-ad/src/NonsmoothFwdAD.jl")

using .NonsmoothFwdAD
using .GeneralizedDiff
using .ConvexOptimization

using JuMP, Ipopt, LinearAlgebra, NLopt, LinearAlgebra

The usage of `NonSmoothFwdAD` is demonstrated by scripts [test.jl](test/test.jl) and [convexTest.jl](test/convexTest.jl). 

#### GeneralizedDiff

Consider the following nonsmooth function of two variables, to replicate Example 6.2 from Khan and Barton (2013):

In [13]:
f(x) = max(min(x[1], -x[2]), x[2] - x[1])

f (generic function with 1 method)

Using the `NonsmoothFwdAD` module (after `include("NonsmoothFwdAD.jl")` and using `.NonsmoothFwdAD`, `.GeneralizedDiff`), we may evaluate a value `y` and a generalized gradient element `yGrad` of `f` at `[0.0, 0.0]` by the following alternative approaches, using the nonsmooth vector forward mode of AD.

- By defining f beforehand:

In [16]:
f(x) = max(min(x[1], -x[2]), x[2] - x[1])
y, yGrad = eval_gen_gradient(f, [0.0, 0.0])

(0.0, [0.0, -1.0])

- By defining f as an anonymous function:

In [17]:
y, yGrad = eval_gen_gradient([0.0, 0.0]) do x
    return max(min(x[1], -x[2]), x[2] - x[1])
end

(0.0, [0.0, -1.0])

Here, `eval_gen_gradient` constructs `yGrad` as a `Vector{Float64}`, and only applies to scalar-valued functions. For vector-valued functions, `eval_gen_derivative` instead produces a generalized derivative element `yDeriv::Matrix{Float64}`.

For scalar-valued functions of one or two variables, the "compass difference" is guaranteed to be an element of Clarke's generalized gradient. We may calculate the compass difference `yCompass::Vector{Float64}` for the above function `f` at `[0.0, 0.0]` as follows:

In [18]:
_, yCompass = eval_compass_difference([0.0, 0.0]) do x
    return max(min(x[1], -x[2]), x[2] - x[1])
end	

(0.0, [-0.5, 0.5])

#### ConvexOptimization

Consider the following optimization problem, replicating Example 6 in F. Facchinei et. al (2014): 

```
min PHI(x) = (x[1] + x[2])*x[4] + 0.5*(x[2] + x[3])^2 
    s.t.    x[1] <= 0
            x[2] >= 1
            x[4] >= 0
            x[1] + x[2] + x[3] >=0 
```

The provided non-smooth reformulation to the Karush-Kuhn-Tucker system is as follows: 

In [19]:
uOffset = 4
vOffset = 9
f(x) = [x[4] + x[1+uOffset] - x[2+uOffset] - x[5+uOffset],
    x[4] + x[2] + x[3] - x[3+uOffset] - x[5+uOffset],
    x[2] + x[3] - x[5+uOffset],
    x[1] + x[2] - x[4+uOffset],
    x[1] + x[1+vOffset],
    - x[1] + x[2+vOffset],
    1.0 - x[2] + x[3+vOffset],
    - x[4] + x[4+vOffset],
    - x[1] - x[2] - x[3] + x[5+vOffset],
    min(x[1+uOffset], x[1+vOffset]),
    min(x[2+uOffset], x[2+vOffset]),
    min(x[3+uOffset], x[3+vOffset]),
    min(x[4+uOffset], x[4+vOffset]),
    min(x[5+uOffset], x[5+vOffset])] 

f (generic function with 1 method)

Using the `NonsmoothFwdAD` module (after `include("NonsmoothFwdAD.jl")` and using `.NonsmoothFwdAD`, `.GeneralizedDiff`, `.ConvexOptimization`), the LPNewton method can locate the minima of `(x, PHI(x))` by solving `f(x) = 0` given no other binding set constraints. Assume an initial guess of `x0` for `f`.

In [23]:
x0 = [1.0, 4.0, -2.0, 1.0,
    3.0, 3.0, 1.0, 4.0, 1.0,
    0.0, 1.0, 3.0, 1.0, 3.0]
x, _, gamma = LPNewton(f, x0)

([0.0014446680056120448, 3.3190935752354815, -3.3176488093686807, -0.00036358277451839, 2.757540039718657, 2.756452152663851, 0.00017972646636453716, 3.3205442397394664, 0.0007223824920200793, -0.0007223339801287438, 0.0007223339799435811, 2.319087744047366, -0.00018199877686866619, 0.0036118164076717465], [1.921788267500533e-6, 0.00017907413389771755, 0.0007223833747807885, -5.996498372962122e-6, 0.000722334025483301, -0.0007223340256684637, -5.8311881154793355e-6, 0.00018158399764972382, 0.000722382535258941, -0.0007223339801287438, 0.0007223339799435811, 0.00017972646636453716, -0.00018199877686866619, 0.0007223824920200793], 4.992764842277354)

Thus, the solution for `PHI(x)` is `(x = [0.0, 3.32, -3.32, 0.0], PHI(x) = 0.0)`. 

Note that the solution set of the optimization problem `PHI(x)` is `X = {(0, t, −t, 0)|t ≥ 1}`. Different initial guesses `x0` could produce different local optima for `PHI(x)` where `f(x) = 0`. This is just one potential solution. 